In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else)","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc)",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist)",Name of the institute I graduated from last.,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you),Barring my personal details (my name and contact number) shared details can be viewed by andwemet community members.,My religious inclination,Video Introduction
0,Male,32,Mumbai,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,Certified financial analyst CFA,Masters,Health concious due to health conditions in fa...,I give consent,,
1,Male,36,New Delhi,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,Institute of chartered accountants of India,Masters,I like playing sports,I give consent,,
2,Male,32,Bangalore,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),"RRCE, Bangalore",Masters,No,I give consent,,
3,Female,33,Gurugram,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,PGI Rohtak,Phd and equivalent,I take time to open up and quite curious,I give consent,,
4,Female,32,Mumbai,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,XIC,PG Diploma,I believe honest communication and mutual resp...,I give consent,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,Male,31,Mumbai,"Having a supportive family, dependable friends...",There were ups & downs but at the end it was a...,Never married,No,Under 12 Lakhs,Sales & Partnership Manager,Mumbai University,Masters,1. I am looking for someone who is a working p...,I give consent,Jain,
156,Female,42,Delhi,"Having supportive, loving family, friends, a c...",I had a lovely childhood growing up in delhi,Was married earlier,No,Between 50-70 Lakhs,Entrepreneur,IIM Kozhikode,PG Diploma,Looking for a life partner to grow and build a...,I give consent,Hindu,
157,Male,29,Mumbai,Having understanding and supporting family,It was challenging and struggling but with fam...,Never married,No,Between 12-15 Lakhs,I am a project manager,NIT JAMSHEDPUR,Undergraduate,I am someone who can listen you 24*7 and alway...,I give consent,Hindu,
158,Female,29,Gurgaon,"Both what I am born with, and what I have crea...",Growing up was a mixed journey. While I had a ...,Never married,No,Between 30-35 lakhs,Consultant; Aspiring Coach & Entrepreneur,Indian School of Business,Masters,I love music - I'm a self-proclaimed percussio...,I give consent,Hindu,


In [3]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else)","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc)",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist)",Name of the institute I graduated from last.,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you),Barring my personal details (my name and contact number) shared details can be viewed by andwemet community members.,My religious inclination,Video Introduction
2,Male,32,Mumbai,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,Certified financial analyst CFA,Masters,Health concious due to health conditions in fa...,I give consent,,
3,Male,36,New Delhi,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,Institute of chartered accountants of India,Masters,I like playing sports,I give consent,,
4,Male,32,Bangalore,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),"RRCE, Bangalore",Masters,No,I give consent,,
5,Female,33,Gurugram,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,PGI Rohtak,Phd and equivalent,I take time to open up and quite curious,I give consent,,
6,Female,32,Mumbai,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,XIC,PG Diploma,I believe honest communication and mutual resp...,I give consent,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,Male,31,Mumbai,"Having a supportive family, dependable friends...",There were ups & downs but at the end it was a...,Never married,No,Under 12 Lakhs,Sales & Partnership Manager,Mumbai University,Masters,1. I am looking for someone who is a working p...,I give consent,Jain,
158,Female,42,Delhi,"Having supportive, loving family, friends, a c...",I had a lovely childhood growing up in delhi,Was married earlier,No,Between 50-70 Lakhs,Entrepreneur,IIM Kozhikode,PG Diploma,Looking for a life partner to grow and build a...,I give consent,Hindu,
159,Male,29,Mumbai,Having understanding and supporting family,It was challenging and struggling but with fam...,Never married,No,Between 12-15 Lakhs,I am a project manager,NIT JAMSHEDPUR,Undergraduate,I am someone who can listen you 24*7 and alway...,I give consent,Hindu,
160,Female,29,Gurgaon,"Both what I am born with, and what I have crea...",Growing up was a mixed journey. While I had a ...,Never married,No,Between 30-35 lakhs,Consultant; Aspiring Coach & Entrepreneur,Indian School of Business,Masters,I love music - I'm a self-proclaimed percussio...,I give consent,Hindu,


In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,5])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
157,Male,31,Mumbai,Never married
158,Female,42,Delhi,Was married earlier
159,Male,29,Mumbai,Never married
160,Female,29,Gurgaon,Never married


In [6]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [8]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [9]:
encoded_categorical_features

,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,Widow/ Widower
2,0,1,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,0,0,0,1,0
6,1,0,0,0,0
...,...,...,...,...,...
157,1,0,0,0,0
158,0,0,0,1,0
159,1,0,0,0,0
160,1,0,0,0,0


In [10]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [11]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
157,Male,31,Mumbai,Never married
158,Female,42,Delhi,Was married earlier
159,Male,29,Mumbai,Never married
160,Female,29,Gurgaon,Never married


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [13]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
157,Male,31,Mumbai,Never married
158,Female,42,Delhi,Was married earlier
159,Male,29,Mumbai,Never married
160,Female,29,Gurgaon,Never married


In [14]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [15]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [16]:
print(features.shape)

(160, 43)


In [17]:
k =100
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=100)

In [18]:
row =9
selectedRow = refDataset.iloc[row]

In [19]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[  9 152 146  22  33 147  89 140 129   2  69  96  68  17  18  15 137  70
   57 154  16  30  93   5 115  27  14  10 132 133  46 130 153  40  49  85
   83 101  75 107  60  44 103   1  97 119  53  35   7 118 155 151 141  63
   38 149 143 150  81  91 157 128  66 121  77  73  50 105  99  48 114 122
    6 145 127   4 106 131  34 117   8 144  94  79 158  98  71  87  47  25
  159   0  82  52  20 125 148  67  28  90]]


In [20]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Female' 35 'Bangalore ' 'Never married']


['Female' 35 'Bangalore' 'Never married']


['Female' 34 'Bangalore' 'Never married']


['Female' 33 'Bangalore ' 'Never married']


['Female' 33 'Bangalore' 'Never married']


['Female' 31 'Bangalore ' 'Never married']


['Female' 31 'Bangalore' 'Never married']


['Female' 30 'Bangalore' 'Never married']


['Female' 29 'Bangalore/ Chennai ' 'Never married']


['Female' 42 'Bangalore ' 'Never married']


['Female' 33 'Delhi' 'Never married']


['Female' 33 'Mumbai' 'Never married']


['Female' 33 'Gurgaon' 'Never married']


['Female' 32 'Mumbai' 'Never married']


['Female' 32 'Delhi' 'Never married']


['Female' 34 'Chicago, IL' 'Never married']


['Female' 34 'Bangalore' 'Was married earlier']


['Female' 34 'Gurgaon' 'Never married']


['Female' 34 'Bikaner ' 'Never married']


['Female' 31 'Gurugram' 'Never married']


['Female' 35 'Gurugram' 'Never married']


['Female' 35 'Hyderabad ' 'Never married']


['Female' 29 'Gurgaon' 'Never 

In [ ]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,5])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [ ]:
print(preprocessData(refDataset))

In [21]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']